In [1]:
import math
from typing import List

import matplotlib.pyplot as plt
import numpy as np


class Point():
    """Data structure of a point."""

    def __init__(self, x, y, z=0) -> None:
        """Point object

        Args:
            x (float): x coordinate of point
            y (float): y coordinate of point
        """
        self.x = x
        self.y = y
        self.z = z

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.z

    def __len__(self):
        return 3

    def __call__(self) -> tuple:
        return (self.x, self.y, self.z)

    def __str__(self) -> str:
        return str((self.x, self.y, self.z))

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def to_numpy(self):
        return np.array([self.x, self.y])


class Edge:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2

    def __eq__(self, other):
        # return self.__dict__ == other.__dict__
        if self.p1 == other.p1 or self.p1 == other.p2:
           if self.p2 == other.p1 or self.p2 == other.p2:
               return True
        return False
    
    def __str__(self):
        return "[" + str(self.p1) + " -> " + str(self.p2) + "]"

    def __call__(self):
        # return "[" + str(self.p1) + " -> " + str(self.p2) + "]"
        return (self.p1(), self.p2())


class Face:
    def __init__(self, e1, e2, e3):
        self.e1 = e1
        self.e2 = e2
        self.e3 = e3

    def __str__(self):
        return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"

    def __call__(self):
        # return "{" + str(self.e1) + ", " + str(self.e2) + ", " + str(self.e3) + "}"
        return self.e1.p1(), self.e1.p2(), self.e3.p1()

    def __eq__(self, other):
        if self.e1 == other.e1 or self.e1 == other.e2 or self.e2 == other.e3:
            if self.e2 == other.e1 or self.e2 == other.e2 or self.e2 == other.e3:
                if self.e3 == other.e1 or self.e3 == other.e2 or self.e3 == other.e3:
                    return True
        return False
        # p1, p2, p3 = self()
        # p1_, p2_, p3_ = other()

        # if p1 == p1_ or p1 == p2_ or p1 == p3_:
        #     if p2 == p1_ or p2 == p2_ or p2 == p3_:
        #         if p3 == p1_ or p3 == p2_ or p3 == p3_:
        #             return True
        # return False


class Gcrf():

    def gen_poly(self, n):
        xn = np.random.random(n)
        yn = np.random.random(n)
        z = zip(xn, yn)
        z = [Point(x, y) for x, y in z]
        z = np.array(z)
        angles = [self.pseudo_angulo_orientado(i) for i in z]
        indexes = np.argsort(angles)
        z = z[indexes]
        z = z.tolist()

        return z

    def soma_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x+b.x, a.y+b.y, a.z+b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]+b[i])
        return z

    def subtr_vetorial(self, a, b):
        if isinstance(a, Point):
            return Point(a.x-b.x, a.y-b.y, a.z-b.z)
        z = []
        for i in range(len(a)):
            z.append(a[i]-b[i])
        return z

    def multi_escalar(self, lamb: int, x: list) -> list:
        # if isinstance(a, Point):
        #     return Point(a.x*lamb, a.y*lamb, a.z*lamb)
        z = []
        for i in range(len(x)):
            z.append(lamb*x[i])
        return z

    def prod_escalar(self, a, b):
        aIsPoint = isinstance(a, Point)
        bIsPoint = isinstance(b, Point)
        if aIsPoint and bIsPoint:
            return sum([a.x*b.x, a.y*b.y, a.z*b.z])
        if aIsPoint:
            a = a()
        if bIsPoint:
            b = b()
        res = []
        for i in range(len(a)):
            res.append(a[i]*b[i])
        return sum(res)

    def norma(self, a):
        norm = a.x**2 + a.y**2 + a.z**2
        return math.sqrt(norm)

    def distancia(self, x, y):
        """Returns the distance between vectors x and y.

        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.

        Returns:
            [float]: [the distance between vectors x and y]
        """
        z = []
        for i in range(len(x)):
            z.append(x[i]-y[i])
        return self.norma(z)

    def angulo(self, x, y, degrees=False):
        """Returns the angle between vectors x and y.

        Args:
            x (list): list containing points from vector x.
            y (list): list containing points from vector y.
            degrees (bool): if the result should be in degrees or radians, defaults to False.
        Returns:
            float: the angle between the two vectors
        """

        # if not degrees:
        #     return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        # return math.acos(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))*(180/math.pi)
        if not degrees:
            return self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))
        return self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))*(180/math.pi)

    def angulo_orientado(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*math.acos(x[0]/self.norma(x))
        return mult*math.acos(x[0]/self.norma(x))*(180/math.pi)

    def pseudo_angulo(self, x, degrees=False):
        if isinstance(x, Point):
            x = x()
        mult = 1
        if x[1] < 0:
            mult = -1
        if not degrees:
            return mult*(1-(x[0]/self.norma(x)))
        return mult*(1-(x[0]/self.norma(x)))*(180/math.pi)

    def pseudo_angulo_cosseno(self, x, y=[0, 0], degrees=False):
        if isinstance(x, Point):
            x = x()
        if not degrees:
            return 1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y)))
        return (1-(self.prod_escalar(x, y)/(self.norma(x)*self.norma(y))))*(180/math.pi)

    def pseudo_angulo_orientado(self, x):
        if isinstance(x, Point):
            x = x()
        if x[1] >= 0:
            if x[0] >= 0:
                if x[0] >= x[1]:
                    return x[1]/x[0]
                return 2-(x[0]/x[1])
            if -x[0] <= x[1]:
                return 2+(-x[0]/x[1])
            return 4-(x[1]/-x[0])
        if x[0] < 0:
            if -x[0] >= -x[1]:
                return 4+(-x[1]/-x[0])
            return 6-(-x[0]/-x[1])
        if x[0] <= -x[1]:
            return 6+(x[0]/-x[1])
        return 8-(-x[1]/x[0])

    def pseudo_angulo_2(self, x, y):
        return self.pseudo_angulo_orientado(y)-self.pseudo_angulo_orientado(x)

    def prod_vetorial(self, a, b):
        if len(a) == 2:
            return a.x*b.y-a.y*b.x
        return Point(a.y*b.z-a.z*b.y, a.z*b.x-a.x*b.z, a.x*b.y-a.y*b.x)

    def intersect(self, a, b, c, d):
        ab = self.subtr_vetorial(b, a)
        ac = self.subtr_vetorial(c, a)
        ad = self.subtr_vetorial(d, a)
        ca = self.subtr_vetorial(a, c)
        cb = self.subtr_vetorial(b, c)
        cd = self.subtr_vetorial(d, c)

        # r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad)
        r1 = self.prod_vetorial(ab, ac)*self.prod_vetorial(ab, ad) < 0
        # r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb)
        r2 = self.prod_vetorial(cd, ca)*self.prod_vetorial(cd, cb) < 0
        # return (r1, r2), r1 < 0 and r2 < 0
        return r1 and r2

    def square_area(self, p1: Point, p2: Point, p3: Point):
        res = self.prod_vetorial(self.subtr_vetorial(
            p2, p1), self.subtr_vetorial(p3, p1))
        res = sum([i**2 for i in res])
        return 0.5*res

    def area(self, x, y):
        if len(x) == 2:
            return abs(self.prod_vetorial(x, y))
        return self.norma(self.prod_vetorial(x, y))

    def signed_volume(self, p1: Point, p2: Point, p3: Point, p4: Point):
        v1 = self.subtr_vetorial(p2, p1)
        v2 = self.subtr_vetorial(p3, p1)
        v3 = self.subtr_vetorial(p4, p1)
        res = self.prod_vetorial(v1, v2)
        return self.prod_escalar(v3, res)/6

    def antiHorario(self, listaDePontos):
        res = 0
        for i in range(len(listaDePontos)-1):
            res += self.prod_vetorial(listaDePontos[i], listaDePontos[i+1])
        res += self.prod_vetorial(listaDePontos[0], listaDePontos[-1])
        return 0.5*res > 0

    def slope(self, p1: Point, p2: Point):
        return (p2.y - p1.y) / (p2.x - p1.x)

    def pip(self, p: Point, poly) -> bool:
        """ Determine if the point is in the polygon.

        Args:
            p -- a object of type Point(x, y)
            poly -- a list of tuples [(x, y), (x, y), ...]

        Returns:
            True if the point is in the path or is a corner or on the boundary"""

        num = len(poly)
        j = num - 1
        result = False
        for i in range(len(poly)):
            if (p.x == poly[i].x) and (p.y == poly[i].y):
                return True
            if ((poly[i].y > p.y) != (poly[j].y > p.y)):
                slope = self.slope(p, poly[i])
                if slope == 0:
                    return True
                if (slope < 0) != (poly[j].y < poly[i].y):
                    result = not result
            j = i
        return result

    def starPolygon(self, points, jumps):
        i = 0
        starredPolygon = []
        while i < jumps:
            for j in range(i, len(points), jumps):
                # print("Ponto adicionado j:",points[j]())
                starredPolygon.append(points[j])
            # print("Ponto adicionado i:",points[i]())
            starredPolygon.append(points[i])
            i += 1
        return starredPolygon

    def centeroidnp(self, points):
        length = len(points)
        tipoPoints = type(points[0])

        if tipoPoints == tuple:
            arr = np.array(points)
        elif tipoPoints == Point:
            arr = np.array([p.to_numpy() for p in points])

        sum_x = np.sum(arr[:, 0])
        sum_y = np.sum(arr[:, 1])
        return sum_x/length, sum_y/length

    def plotPoint(self, p: Point):
        plt.scatter(p.x, p.y)
        plt.show()

    def plotPoints(self, p: List[Point]):
        for point in p:
            self.plotPoint(point)
        plt.show()

    def plotPolygon(self, points, annotate=False):
        """Plot polygon from list of points

        Args:
            points (Point): list of Point objects indicating the polygon vertices
        """

        points.append(points[0])
        if type(points[0]) == Point:
            points = [p() for p in points]
        x, y = zip(*points)
        plt.grid(0.5)
        plt.plot(x, y, '-o')
        if annotate:
            plt.text(x[0], y[0], "p1")
            plt.text(x[1], y[1], "p2")
            plt.text(x[2], y[2], "p3")
        plt.show()

    def plotSegment(self, segments: List[Point], threeD=False) -> None:
        if threeD:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

        for segment in segments:
            segx = [p.x for p in segment]
            segy = [p.y for p in segment]
            if threeD:
                segz = [p.z for p in segment]
                ax.plot(segx, segy, segz)
            else:
                plt.plot(segx, segy)
        plt.show()

    def plotSquare(self):
        x = [1, 1, -1, -1, 1]
        y = [1, -1, -1, 1, 1]
        plt.plot(x, y)

    def plotAxis(self):
        plt.grid()
        plt.axhline(y=0, color='k')
        plt.axvline(x=0, color='k')


    # def plot_faces(self, F):


    def rotationIndex(self, p: Point, poly: list):
        # NOT WORKING
        j = len(poly)-1
        rotIndex = 0
        for i in range(len(poly)):
            ppi = self.subtr_vetorial(p, poly[j])
            ppi1 = self.subtr_vetorial(p, poly[i])
            rotIndex += self.pseudo_angulo_2(ppi, ppi1)
            j = i
        rotIndex *= 1/(2*math.pi)
        if rotIndex == 0:
            return False
        if abs(rotIndex) == 1:
            return True
        return rotIndex


In [2]:
import os
import math
# from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [3]:
gc = Gcrf()

In [4]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [5]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [6]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [7]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [8]:
p1 = Point(0,-1,0)
p2 = Point(0, 0, 0)
p3 = Point(0,0,1)
a = Point(1, 0,0)
b = Point(-1,0,0)
print(gc.signed_volume(p1,p2,p3,a))
print(gc.signed_volume(p1,p2,p3,b))

0.16666666666666666
-0.16666666666666666


In [37]:
def PivotAroundEdge(f, e, P):
    minAngle = 2*math.pi
    candidate = None
#     print("E",e, "F",f)
    n1 = normal(f.e1.p1, f.e1.p2, f.e2.p2)
    
    for i in range(len(P)):
        if P[i] == e.p1 or P[i] == e.p2: continue
#         print("PONTOS DA FACE",e.p2, e.p1, P[i])
        n2 = normal(e.p2, e.p1, P[i])
#         print("n1", n1, "n2", n2)
#         if n2 == Point(0,0,0): continue
        angle = -1*gc.angulo(n1,n2)
#         angle = gc.angulo(n1,n2)
#         print("Angulo",angle)
        if angle < minAngle and P[i] != e.p1 and P[i] != e.p2:
            minAngle = angle
            candidate = i
    return P[candidate]

In [10]:
def FreeEdge(F, e):
    count = 0
    for f in F:
        if (f.e1 == e or f.e2 == e or f.e3 == e):
            count+=1
#     print("Count:",count)
    if count > 1:
        return False
    return True

In [11]:
def FindFreeEdge(F, face):
    free_edges = []
    for face in F:
#         print("FACE no FIND:", face)
        if(FreeEdge(F, face.e1)):
#             print("TRUE no IF 1")
            free_edges.append(face.e1)
        if(FreeEdge(F, face.e2)):
#             print("TRUE no IF 2")
            free_edges.append(face.e2)
        if(FreeEdge(F, face.e3)):
#             print("TRUE no IF 3")
            free_edges.append(face.e3)
#     print("FREE_EDGES:", free_edges)
    return free_edges

In [12]:
# def PivotAroundEdge(e, P):
#     minAngle = 2*math.pi
#     candidate = None
#     normal = gc.prod_vetorial(e.p1, e.p2)
# #     print("NORMAL:", normal)
#     for i in range(len(P)):
#         print("P[i]",P[i])
#         angle = gc.angulo(normal, P[i])
#         if angle < minAngle:
#             minAngle = angle
#             candidate = i
#     return P[candidate]

In [13]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [14]:
def normal(p1, p2, p3):
    v1 = gc.subtr_vetorial(p1,p2) 
#     print("v1",v1)
    v2 = gc.subtr_vetorial(p1,p3)
#     print("v2",v2)
    return gc.prod_vetorial(v1,v2)

In [15]:
def find_first_face(P):
    minAngle = 2*math.pi
    candidate = None
    
    p1 = BottomMostLeftMostBackMost(P)
    p2 = Point(p1.x+1, p1.y, p1.z)
    p3 = Point(p1.x, p1.y, p1.z+1) 
#     print("primeiro P1", p1, "P2", p2, "P3", p3)
    n1 = normal(p1, p2, p3)
#     print("n1",n1)
    for i in range(len(P)):
        if P[i] == p1 or P[i] == p2: continue
        n2 = normal(p1, p2, P[i])
#         print("n2",n2)
        angle = gc.angulo(n1,n2)

        if angle <= minAngle:
            minAngle = angle
            candidate = i
        
    p3 = P[candidate]

    minAngle = 2*math.pi
    candidate = None
    
    n1 = normal(p3, p1, p2)
    for j in range(len(P)):
        if P[j] == p1 or P[j] == p3: continue
        n2 = normal(p1, p3, P[j])
        angle = gc.angulo(n1,n2)
        if angle < minAngle:
            minAngle = angle
            candidate = j
    
    p2 = P[candidate]
    
    e1 = Edge(p1,p2)
    e2 = Edge(p2, p3)
    e3 = Edge(p3, p1)
    print("P {} Q {} R {}".format(p1,p2,p3))
    face = Face(e1, e2, e3)
    return face

In [16]:
# {[(0.0, -1.0, -1.0) -> (-0.866025, 1.0, -0.5)], [(-0.866025, 1.0, -0.5) -> (0.0, 1.0, -1.0)], [(0.0, 1.0, -1.0) -> (0.0, -1.0, -1.0)]}
# {[(0.0, -1.0, -1.0) -> (-0.866025, -1.0, -0.5)], [(-0.866025, -1.0, -0.5) -> (0.0, 1.0, -1.0)], [(0.0, 1.0, -1.0) -> (0.0, -1.0, -1.0)]}
p1 = Point(0.000000 ,-1.000000 ,-1.000000) #--- 0 
p2 = Point(0.000000 ,1.000000 ,-1.000000)  #--- 2
p3 = Point(0.866025 ,-1.000000 ,-0.500000) #--- 1
p4 = Point(0.866025 ,1.000000 ,-0.500000)
p5 = Point(0.866025 ,-1.000000 ,0.500000)
p6 = Point(0.866025 ,1.000000 ,0.500000)
p7 = Point(-0.000000, -1.000000, 1.000000)
p8 = Point(-0.000000, 1.000000 ,1.000000)
p9 = Point(-0.866025, -1.000000, 0.500000)
p10 = Point(-0.866025, 1.000000 ,0.500000)
p11 = Point(-0.866025, -1.000000, -0.500000)
p12 = Point(-0.866025, 1.000000 ,-0.500000)

P = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12]

In [17]:
f = find_first_face(P)
print(f)

P (0.0, -1.0, -1.0) Q (0.866025, -1.0, -0.5) R (0.0, 1.0, -1.0)
{[(0.0, -1.0, -1.0) -> (0.866025, -1.0, -0.5)], [(0.866025, -1.0, -0.5) -> (0.0, 1.0, -1.0)], [(0.0, 1.0, -1.0) -> (0.0, -1.0, -1.0)]}


In [18]:
def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [41]:
def GiftWrap(P):
#     t = FindTriangleOnHull(P)
    t = find_first_face(P)
    print("Triangulo encontrado:", t)
#     Q = [t.e1, t.e2, t.e3]
    H = [t]
#     Processed = []
    faces = [t]
#     print("Q possui as arestas:")
#     for q in Q: print(q)
#     while not len(Q) == 0:
    while not len(faces) == 0:
        t = faces.pop(0)
        edges = FindFreeEdge(H, t)
#         print("EDGES:")
#         for e___ in edges: print(e___)
#         e = Q.pop(0)
#         e = Edge(e.p2, e.p1)
        for e in edges:
#             print("Aresta a ser processada:", e)
#             if NotProcessed(Processed, e):
#             print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(t, e, P)
#             print("Resultado do PivotAroundEdge:", q)
            e = Edge(e.p2, e.p1)

            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
#             print("Nova face:", t)
#             H.append(t)

            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
#                 print("{} entrou no Hull".format(t))
            else: print("Face já está no Hull")
#             e_ = Edge(e.p2, e.p1)
#             Q.extend([t.e1, t.e2, t.e3])
#             print("Q agora possui as arestas:")
#             for q in Q: print(q)
#             Processed.append(e)
#             print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
#             for aresta in Processed: print(aresta)
            faces.append(t)
#         else: print("Aresta JÁ PROCESSADA")
        
    return H

In [44]:
# p1 = Point(-3, 0, 0)
# p2 = Point(0, -3, 0)  # --- 0
# p3 = Point(0, 2, 0)
# p4 = Point(0, 0, 3)
# p5 = Point(0, 0, -4)  # ---2
# p6 = Point(4, 0, 0)  # --- 1

p1 = Point(-1, 0, 0)
p2 = Point(0, -1, 0)
p3 = Point(0, 1, 0)
p4 = Point(0, 0, 1)
p5 = Point(0, 0, -1)
p6 = Point(1, 0, 0)
# p7 = Point(0,0,0)

# P = [p1, p2, p3, p4, p5, p6, p7]
P = [p1, p2, p3, p4, p5, p6]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

P (0, -1, 0) Q (1, 0, 0) R (0, 0, -1)
Triangulo encontrado: {[(0, -1, 0) -> (1, 0, 0)], [(1, 0, 0) -> (0, 0, -1)], [(0, 0, -1) -> (0, -1, 0)]}
Face já está no Hull
Face já está no Hull
{[(0, -1, 0) -> (1, 0, 0)], [(1, 0, 0) -> (0, 0, -1)], [(0, 0, -1) -> (0, -1, 0)]}
{[(1, 0, 0) -> (0, -1, 0)], [(0, -1, 0) -> (0, 0, 1)], [(0, 0, 1) -> (1, 0, 0)]}
{[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (1, 0, 0)], [(1, 0, 0) -> (0, -1, 0)]}
{[(0, 0, 1) -> (0, -1, 0)], [(0, -1, 0) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, 0, 1)]}
{[(1, 0, 0) -> (0, 0, 1)], [(0, 0, 1) -> (0, -1, 0)], [(0, -1, 0) -> (1, 0, 0)]}
{[(0, 0, 1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)], [(0, -1, 0) -> (0, 0, 1)]}
6


In [22]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [23]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [24]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
    return points

In [25]:
points = read_vertices_from_obj(r"C:\Users\romul\Documents\UFC\MESTRADO\Disciplinas\Geometria Computacional\super-nintendo-blend\p1.obj")
print(len(points))

348


In [26]:
path = os.path.join(os.getcwd(), 'obj')
if os.path.isdir(path):
    print("OBJ folder exists")
else:
    print("Creating OBJ folder.")
    os.mkdir(path)

OBJ folder exists


In [27]:
# print_faces([find_first_face(points)], points)

In [28]:
from time import time

In [29]:
initial_time = time()
print("Initial Time:", initial_time)
nH = GiftWrap(points)
print("Demorou: {}".format(time()-initial_time()))
# for h in nH:
#     print(h)
print(len(nH))

Initial Time: 1627261496.0621986
P (-0.258873, 0.032391, 0.264508) Q (-0.340916, 0.033424, 0.136366) R (-0.346055, 0.032391, 0.129129)
Triangulo encontrado: {[(-0.258873, 0.032391, 0.264508) -> (-0.340916, 0.033424, 0.136366)], [(-0.340916, 0.033424, 0.136366) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.258873, 0.032391, 0.264508)]}


KeyboardInterrupt: 

In [45]:
# print_faces(H, P)
write_faces(H, P, path+"/bento2.obj")

In [ ]:
write_faces(nH, points, path+"/controle.obj")
# print_faces(nH, points)


In [ ]:
points = read_vertices_from_obj("/Users/romul/Documents/UFC/MESTRADO/Disciplinas/Geometria Computacional/obj/esfera.obj")
print(len(points))

In [ ]:
find_first_face(points)